In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt

In [ ]:
aug_train = pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv')  
aug_test = pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_test.csv') 

In [ ]:
aug_train.shape

In [ ]:
aug_train.head()

In [ ]:
aug_train.drop(['enrollee_id'], axis = 1, inplace = True)

In [ ]:
string_cols = ['city','gender','relevent_experience','enrolled_university','education_level','major_discipline','company_type']

In [ ]:
aug_train.isnull().mean()*100

In [ ]:
aug_test.isnull().mean() * 100

In [ ]:
aug_train.groupby('company_type').agg({'target': ['sum', 'count','mean']})

In [ ]:
aug_train.groupby('gender').agg({'target': ['sum', 'count','mean']})

In [ ]:
for i in  aug_train.isna().mean()[ aug_train.isna().mean() > 0].index:
    print(i,  aug_train[i].unique(), '\n')

In [ ]:
for i in  aug_train.columns:
    print(i,  aug_train[i].unique(),'\n')

# Feature Engineering

In [ ]:
aug_train.loc[aug_train['experience']== '>20','experience'] = '20'
aug_train.loc[aug_train['experience']== '<1','experience'] = '1'
aug_train['experience'].fillna('0',inplace = True)
aug_train['experience'] = aug_train['experience'].astype('int32')

In [ ]:
aug_train.loc[aug_train['last_new_job']== '>4','last_new_job'] = '5'
aug_train.loc[aug_train['last_new_job']== 'never','last_new_job'] = '0'
aug_train['last_new_job'].fillna('-1',inplace = True)
aug_train['last_new_job'] = aug_train['last_new_job'].astype('int32')

In [ ]:
aug_train['company_size'] = aug_train['company_size'].map({'0':0,
                               '50-99':50,
                               '<10': 1,
                               '10000+':10000,
                               '5000-9999':5000,
                               '1000-4999':1000,
                               '10/49':10,
                               '100-500':100,
                               '500-999':500})
aug_train['company_size'].fillna(-1,inplace = True)
aug_train['company_size'] = aug_train['company_size'].astype('int32')

In [ ]:
aug_train['city'] = aug_train['city'].str.replace('city_', '', regex=False).astype('int32')

In [ ]:
for col in string_cols:
    aug_train[col].fillna('NaN',inplace = True)

In [ ]:
for i in  aug_train.columns:
    print(i,  aug_train[i].unique(),'\n')

# Nan imputation

In [ ]:
aug_train.isnull().mean()*100

In [ ]:
#msno.heatmap(aug_train)

In [ ]:
aug_train.agg({'target': ['sum','mean']})

In [ ]:
aug_train.describe()

In [ ]:
aug_train.dtypes

In [ ]:
aug_train.dtypes.value_counts()

In [ ]:
lst_col_target = 'target'

In [ ]:
def encode_and_bind(original_dataframe, feature_to_encode):
    #dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    #res = pd.concat([original_dataframe, dummies], axis=1)
    res = pd.get_dummies(original_dataframe,columns=feature_to_encode,drop_first=True)
    return(res)

In [ ]:
str_cols = list(aug_train.dtypes.index[aug_train.dtypes == 'O'])

In [ ]:
str_cols

In [ ]:
X_endc = encode_and_bind(aug_train, str_cols)

In [ ]:
X_endc.head()

In [ ]:
X_endc.dtypes.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_endc.drop(columns=['target']),
                                                    X_endc['target'], 
                                                    test_size=0.25, 
                                                    random_state=42)

In [ ]:
X_train.sample(5)

# **Decision Tree**

In [ ]:
from sklearn import tree
dct = tree.DecisionTreeClassifier(max_depth=5,min_samples_leaf=30)
dct.fit(X_train,y_train)

In [ ]:
#tree.plot_tree(dct)

In [ ]:
import graphviz 
dot_data = tree.export_graphviz(dct, out_file=None,filled=True, rounded=True, )  
graph = graphviz.Source(dot_data)  
#graph 

In [ ]:
graph

In [ ]:
graph.render('dct', view=False) 

In [ ]:
y_pred_lr = dct.predict(X_test)
y_pred_proba = dct.predict_proba(X_test)[:,1]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_lr)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_lr))

# **Random Forrest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=4, random_state=0)

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
y_pred_lr = clf.predict(X_test)

In [ ]:
y_pred_proba = clf.predict_proba(X_test)[:,1]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_lr)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_lr))

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score, plot_confusion_matrix
from sklearn.metrics import plot_roc_curve, plot_precision_recall_curve, classification_report, precision_score, recall_score

In [ ]:
df_report = pd.DataFrame(columns={'ROC-AUC'}, data=[0])
df_report['ROC-AUC'] = roc_auc_score(y_test, y_pred_proba)
df_report['F1'] = f1_score(y_test, y_pred_lr)
df_report['precision_0'] = precision_score(y_test, y_pred_lr, pos_label=0)
df_report['recall_0'] = recall_score(y_test, y_pred_lr, pos_label=0)
df_report['precision_1'] = precision_score(y_test, y_pred_lr, pos_label=1)
df_report['recall_1'] = recall_score(y_test, y_pred_lr, pos_label=1)

df_report.index = ['LogisticRegression']

In [ ]:
df_report

In [ ]:
score = pd.DataFrame({'score':y_pred_proba, 'target':y_test})

In [ ]:
score.sample(5)

In [ ]:
score['pct_rank'] = (score['score'].rank(pct=True)*10).astype('int')

In [ ]:
score.describe()

In [ ]:
score.sample(5)

In [ ]:
score['score'] = score['score'].astype('float')
score['bin'] = pd.qcut(score['score'].rank(method='first'), 10, labels=['{}'.format(str(x+1).zfill(2)) for x in range(10)]) 

In [ ]:
Lift_chart = score[score.target==1].groupby('bin').count()

In [ ]:
Lift_chart

In [ ]:
Lift_chart['pcnt'] = Lift_chart.iloc[:,1]/score.groupby('bin').count().iloc[:,0]*100

In [ ]:
Lift_chart

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.countplot(score[score.target==1]['bin'])